In [7]:
import mido

In [8]:
mido.MidiFile('./ACGrand.mid')

MidiFile(type=0, ticks_per_beat=96, tracks=[
  MidiTrack([
    MetaMessage('track_name', name='Acoustic Grand Piano', time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=36, notated_32nd_notes_per_beat=8, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=7, value=127, time=0),
    Message('control_change', channel=0, control=10, value=64, time=0),
    Message('note_on', channel=0, note=38, velocity=110, time=0),
    Message('note_on', channel=0, note=76, velocity=110, time=0),
    Message('note_off', channel=0, note=38, velocity=0, time=46),
    Message('note_off', channel=0, note=76, velocity=0, time=0),
    Message('note_on', channel=0, note=38, velocity=110, time=2),
    Message('note_on', channel=0, note=76, velocity=110, time=0),
    Message('note_off', cha

In [9]:
import mido
import pretty_midi
import math
import os
# --- Function to process the MIDI file ---
def process_midi_to_pretty_notes(filepath, target_ppqn, target_bpm, meter):
    """
    Reads a MIDI file, extracts notes, rescales timing to target PPQN and BPM,
    generates a click track, and calculates the start time of the next measure.

    Args:
        filepath (str): Path to the MIDI file.
        target_ppqn (int): The desired Pulses Per Quarter Note for output interpretation.
        target_bpm (float): The desired Beats Per Minute for output interpretation.
        meter (int): The number of beats per measure (e.g., 4 for 4/4 time). Defaults to 4.

    Returns:
        tuple: A tuple containing:
            - list[pretty_midi.Note]: List of notes from the MIDI file.
            - list[pretty_midi.Note]: List of notes representing the click track.
            - float: Total duration of the track in seconds based on target BPM.
            - float: Time offset (in seconds) for the start of the measure
                     immediately following the last event.
    """
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"MIDI file not found: {filepath}")
    if meter <= 0:
        raise ValueError("Meter (beats per measure) must be positive.")
    if target_bpm <= 0:
        print("Warning: Target BPM is zero or negative. Click track and time offset calculation will be skipped.")
        # Allow processing notes, but click/offset is meaningless
        # target_bpm = 0 # Or raise error? For now, proceed but skip calcs.

    notes_list = []
    click_track_list = []
    active_notes = {}  # Store start times (in target ticks) of active notes: {(channel, pitch): start_tick}
    max_time_sec = 0.0 # Track the timestamp of the *end* of the last note or event

    try:
        mid = mido.MidiFile(filepath)
    except Exception as e:
        raise IOError(f"Could not read MIDI file {filepath}: {e}")

    original_ppqn = mid.ticks_per_beat
    if not original_ppqn or original_ppqn <= 0:
        print(f"Warning: MIDI file '{filepath}' has invalid ticks_per_beat ({original_ppqn}). Assuming 96.")
        original_ppqn = 96

    ppqn_ratio = float(target_ppqn) / original_ppqn if original_ppqn else 0 # Avoid division by zero
    target_tempo_usec = mido.bpm2tempo(target_bpm) if target_bpm > 0 else 0

    print(f"Processing '{filepath}'...")
    print(f"Original PPQN: {original_ppqn}, Target PPQN: {target_ppqn}, Target BPM: {target_bpm}, Meter: {meter}/4") # Assuming /4 for print
    print(f"PPQN Ratio: {ppqn_ratio:.4f}, Target Tempo (usec/beat): {target_tempo_usec}")

    current_tempo_usec = 500000 # Default MIDI tempo (120 BPM) if none found early
    # Find initial tempo if set in track 0
    if mid.type == 1 and len(mid.tracks) > 0:
         for msg in mid.tracks[0]:
             if msg.type == 'set_tempo':
                 current_tempo_usec = msg.tempo
                 print(f"Found initial tempo in track 0: {mido.tempo2bpm(current_tempo_usec):.2f} BPM")
                 break
    # If no set_tempo found, calculate tempo from target_bpm for conversion
    if current_tempo_usec == 500000 and target_bpm > 0:
         current_tempo_usec = target_tempo_usec # Use target tempo if no initial tempo found

    for i, track in enumerate(mid.tracks):
        print(f"--- Processing Track {i} ---")
        current_time_original_ticks = 0

        for msg in track:
            current_time_original_ticks += msg.time

            # Handle tempo changes within the track if necessary (more complex)
            # For this version, we assume a constant target tempo for conversion
            # if msg.type == 'set_tempo':
            #     current_tempo_usec = msg.tempo # Update tempo if it changes mid-track

            # Rescale time to target ticks
            current_time_target_ticks = round(current_time_original_ticks * ppqn_ratio) if ppqn_ratio else 0

            # Convert target ticks to seconds using target PPQN and TARGET BPM/Tempo
            current_time_sec = 0.0
            if target_ppqn > 0 and target_tempo_usec > 0:
                 current_time_sec = mido.tick2second(current_time_target_ticks, target_ppqn, target_tempo_usec)

            note_key = None
            is_note_on = False
            is_note_off = False
            pitch = -1
            velocity = 0

            if msg.type == 'note_on' and msg.velocity > 0:
                is_note_on = True
                pitch = msg.note
                velocity = msg.velocity
                note_key = (msg.channel, pitch)
                max_time_sec = max(max_time_sec, current_time_sec) # Note start updates max time

            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                is_note_off = True
                pitch = msg.note
                note_key = (msg.channel, pitch)
                # Note off time is the crucial one for overall duration
                max_time_sec = max(max_time_sec, current_time_sec)

            elif msg.is_meta: # Other meta messages might indicate end of track
                 max_time_sec = max(max_time_sec, current_time_sec)


            # --- Handle Note On/Off Logic (modified slightly for clarity) ---
            if is_note_on:
                if note_key in active_notes:
                    # Overlap: Close previous note at the start of the new one
                    start_tick_prev, start_sec_prev, vel_prev = active_notes.pop(note_key)
                    end_sec_prev = current_time_sec
                    if end_sec_prev > start_sec_prev:
                        notes_list.append(pretty_midi.Note(velocity=vel_prev, pitch=pitch, start=start_sec_prev, end=end_sec_prev))
                        # print(f"Warning: Note On overlap {note_key} at {current_time_sec:.3f}s. Closing previous.")

                # Store note on info: start tick, start sec, velocity
                active_notes[note_key] = (current_time_target_ticks, current_time_sec, velocity)

            elif is_note_off:
                if note_key in active_notes:
                    start_tick, start_sec, vel = active_notes.pop(note_key)
                    end_sec = current_time_sec

                    if end_sec > start_sec:
                        pm_note = pretty_midi.Note(
                            velocity=vel, # Use stored velocity from note_on
                            pitch=pitch,
                            start=start_sec,
                            end=end_sec
                        )
                        notes_list.append(pm_note)
                    # else: Warn about zero duration if needed
                # else: Warn about note off for inactive note if needed

        # After processing a track, clear any remaining active notes (notes held until end)
        # Use the final max_time_sec as their end time
        keys_to_clear = list(active_notes.keys())
        for note_key in keys_to_clear:
             pitch = note_key[1] # Get pitch from key
             start_tick, start_sec, vel = active_notes.pop(note_key)
             end_sec = max_time_sec # End note at the very end of the track content
             if end_sec > start_sec:
                 print(f"Note {note_key} was still active at end of track. Closing at {max_time_sec:.3f}s.")
                 notes_list.append(pretty_midi.Note(velocity=vel, pitch=pitch, start=start_sec, end=end_sec))


    # --- Calculate Time Offset for Next Measure ---
    time_offset = 0.0
    if target_bpm > 0 and meter > 0:
        seconds_per_beat = 60.0 / target_bpm
        seconds_per_measure = seconds_per_beat * meter

        if seconds_per_measure > 0:
            # Find the end time of the measure containing the last event
            # Ceiling division gives the index of the measure *after* the last event
            measure_index_after_last = math.ceil(max_time_sec / seconds_per_measure)

            # The offset is the start time of that next measure
            time_offset = measure_index_after_last * seconds_per_measure

            # Handle tiny floating point inaccuracies near measure boundaries
            # If max_time_sec is very close to a measure boundary, ceiling might push it
            # unnecessarily. Let's add a small epsilon check.
            epsilon = 1e-9
            measure_boundary_time = (measure_index_after_last -1) * seconds_per_measure
            if max_time_sec > measure_boundary_time - epsilon and max_time_sec <= measure_boundary_time + epsilon:
                 # If max_time_sec is essentially AT the previous measure boundary, recalculate
                 measure_index_containing_last = math.floor(max_time_sec / seconds_per_measure)
                 time_offset = (measure_index_containing_last + 1) * seconds_per_measure


    # --- Generate Click Track ---
    if target_bpm > 0 and meter > 0:
        seconds_per_beat = 60.0 / target_bpm
        # Calculate number of beats up to the END of the measure containing the last note
        # This ensures the click track covers the full final measure.
        num_beats_total = math.ceil(time_offset / seconds_per_beat) if seconds_per_beat > 0 else 0

        print(f"\nEffective duration for clicks/offset: {time_offset:.2f} seconds")
        print(f"Generating click track ({num_beats_total} beats at {target_bpm} BPM, meter={meter})...")

        for i in range(num_beats_total): # Iterate beat by beat
            click_start_time = i * seconds_per_beat
            click_end_time = click_start_time + 0.05

            # Ensure click doesn't exceed the calculated offset time (start of next measure)
            # Although unlikely with short duration, it's good practice.
            click_end_time = min(click_end_time, time_offset)

            if click_end_time > click_start_time:
                # Determine pitch based on position in measure
                if i % meter == 0:
                    pitch = 75 # Beat 1
                else:
                    pitch = 56    # Other beats

                click_note = pretty_midi.Note(
                    velocity=100,
                    pitch=pitch,
                    start=click_start_time,
                    end=click_end_time
                )
                click_track_list.append(click_note)

    return notes_list, click_track_list, max_time_sec, time_offset

In [10]:
process_midi_to_pretty_notes('./ACGrand.mid', 24, 120, 4)

Processing './ACGrand.mid'...
Original PPQN: 96, Target PPQN: 24, Target BPM: 120, Meter: 4/4
PPQN Ratio: 0.2500, Target Tempo (usec/beat): 500000
--- Processing Track 0 ---

Effective duration for clicks/offset: 64.00 seconds
Generating click track (128 beats at 120 BPM, meter=4)...


([Note(start=0.000000, end=0.250000, pitch=38, velocity=110),
  Note(start=0.000000, end=0.250000, pitch=76, velocity=110),
  Note(start=0.250000, end=0.500000, pitch=38, velocity=110),
  Note(start=0.250000, end=0.500000, pitch=76, velocity=110),
  Note(start=0.750000, end=1.000000, pitch=38, velocity=110),
  Note(start=0.750000, end=1.000000, pitch=76, velocity=110),
  Note(start=1.250000, end=1.500000, pitch=38, velocity=110),
  Note(start=1.250000, end=1.500000, pitch=72, velocity=110),
  Note(start=1.500000, end=1.916667, pitch=38, velocity=110),
  Note(start=1.500000, end=1.916667, pitch=76, velocity=110),
  Note(start=2.000000, end=2.416667, pitch=43, velocity=110),
  Note(start=2.000000, end=2.416667, pitch=79, velocity=110),
  Note(start=3.000000, end=3.416667, pitch=47, velocity=110),
  Note(start=3.000000, end=3.416667, pitch=67, velocity=110),
  Note(start=4.000000, end=4.250000, pitch=67, velocity=110),
  Note(start=4.250000, end=4.500000, pitch=74, velocity=110),
  Note(s

In [1]:
import mido
mido.get_input_names(), mido.get_output_names()


(['IAC Driver Bus 1',
  'Launchkey Mini MK3 MIDI Port',
  'Launchkey Mini MK3 DAW Port',
  'Teensy MIDI Port 1'],
 ['IAC Driver Bus 1',
  'Launchkey Mini MK3 MIDI Port',
  'Launchkey Mini MK3 DAW Port',
  'Teensy MIDI Port 1'])

In [9]:
def map_value(input_value, input_min=60, input_max=170, output_min=0, output_max=127):
    scaled_value = ((input_value - input_min) * (output_max - output_min)) / (input_max - input_min) + output_min
    return int(scaled_value)

def reverse_map_value(scaled_value, input_min=60, input_max=170, output_min=0, output_max=127):
    input_value = ((scaled_value - output_min) * (input_max - input_min)) / (output_max - output_min) + input_min
    return input_value

In [20]:
map_value(120)

69

In [18]:
reverse_map_value(map_value(120))

119.76377952755905

In [28]:
import time
import mido

with mido.open_input("Launchkey Mini MK3 MIDI Port") as inport, mido.open_output("Teensy MIDI Port 1") as clock_control:
    while True:
        for port in (inport,):
            for msg in port.iter_pending():
                if msg.type != 'clock':
                    print(msg)
                    msg.channel = 2
        time.sleep(0.01)

note_on channel=15 note=36 velocity=29 time=0
note_on channel=15 note=36 velocity=0 time=0
note_on channel=15 note=24 velocity=126 time=0
note_on channel=15 note=26 velocity=127 time=0
note_on channel=15 note=24 velocity=0 time=0
note_on channel=15 note=27 velocity=124 time=0
note_on channel=15 note=26 velocity=0 time=0
note_on channel=15 note=27 velocity=0 time=0
note_on channel=15 note=29 velocity=127 time=0
note_on channel=15 note=29 velocity=0 time=0
note_on channel=15 note=31 velocity=127 time=0
note_on channel=15 note=31 velocity=0 time=0


KeyboardInterrupt: 